In [1]:
import yaml
import pandas as pd
import numpy as np
from message_ix.utils import make_df
from message_ix.models import MESSAGE_ITEMS
from collections.abc import Mapping
from itertools import repeat

<IPython.core.display.Javascript object>

In [2]:
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)
    
years = [*range(700,730,10)]
regions = ['Westeros', 'Westerlands']
modes = ['standard']
emissions = ['CO2','CH4']
time = ['year']
first_active_year = tech_data['model_data'].get('first_active_year')

In [3]:
years_dict = {yv:[ya for ya in years if ya >= yv] for yv in years if yv >= first_active_year}

vtg = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

act = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

years_vtg_act = pd.DataFrame(
    {
    'year_vtg':[k for k, v in years_dict.items() for i in range(len(v))],
    'year_act':[ya for value in years_dict.values() for ya in value]
    }
)

In [4]:
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

In [5]:
# Basic DataFrame
#group1 = ['inv_cost','technical_lifetime','growth_new_capacity_up','initial_new_capacity_up']
#group2 = ['var_cost','fix_cost','emission_factor','capacity_factor']
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            if parameters[par] == ['node_loc', 'technology', 'year_vtg']:
                kwargs = {'year_vtg': sorted(list(set(years_vtg_act['year_vtg'])))}
            elif all(e in ['node_loc', 'technology', 'year_vtg', 'year_act'] for e in parameters[par]):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                ))
            if 'emission' in parameters[par]:
                data[par] = data[par]*len(emissions)
                for e in range(len(emissions)):
                    data[par][e] = data[par][e].assign(emission=emissions[e])
            if 'mode' in parameters[par]:
                data[par] = data[par]*len(modes)
                for m in range(len(modes)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(mode=modes[m])
            if 'time' in parameters[par]:
                data[par] = data[par]*len(time)
                for t in range(len(time)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(time=time[t])
            

data = {k: pd.concat(v).reset_index(drop=True) for k, v in data.items()}

data

{'emission_factor':    node_loc technology  year_vtg  year_act      mode emission  value      unit
 0      None      DACCS       700       700  standard      CO2      1  tCO2/kWa
 1      None      DACCS       700       710  standard      CO2      1  tCO2/kWa
 2      None      DACCS       700       720  standard      CO2      1  tCO2/kWa
 3      None      DACCS       710       710  standard      CO2      1  tCO2/kWa
 4      None      DACCS       710       720  standard      CO2      1  tCO2/kWa
 5      None      DACCS       720       720  standard      CO2      1  tCO2/kWa
 6      None      DACCS       700       700  standard      CH4      1  tCO2/kWa
 7      None      DACCS       700       710  standard      CH4      1  tCO2/kWa
 8      None      DACCS       700       720  standard      CH4      1  tCO2/kWa
 9      None      DACCS       710       710  standard      CH4      1  tCO2/kWa
 10     None      DACCS       710       720  standard      CH4      1  tCO2/kWa
 11     None      DAC

In [6]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    for tech, diffs in tech_data['model_data'].items():
        if tech != 'first_active_year':
            for reg in regions:
                print(par,tech,reg)
                multiplier = []
                for i in range(len(data[par])):
                    multiplier.append(
                        np.prod([diffs.get(par,{}).get('node_loc',{}).get(reg,1), # by region 
                                 ((1+diffs.get(par,{}).get('year_vtg',{}).get('rate',0))
                                  **(data[par]['year_vtg'][i]-first_active_year)), # by year_vtg
                                 ((1+diffs.get(par,{}).get('year_act',{}).get('rate',0))
                                  **(data.get(par,{}).get('year_act',{}) # by year_act
                                     .get(i,data[par]['year_vtg'][i])-data[par]['year_vtg'][i])),
                                 diffs.get(par,{}).get('mode',{}).get(data.get(par,{}).get('mode',{}).get(i),1), # by mode
                                 diffs.get(par,{}).get('emission',{}).get(data.get(par,{}).get('emission',{}).get(i),1), # by emission
                        ])
                    )

                value = data[par]['value']*multiplier
                data_expand[par].append(
                    data[par].assign(node_loc=reg,value=value)
                   )

data_expand

emission_factor DACCS Westeros
emission_factor DACCS Westerlands
var_cost DACCS Westeros
var_cost DACCS Westerlands
inv_cost DACCS Westeros
inv_cost DACCS Westerlands
fix_cost DACCS Westeros
fix_cost DACCS Westerlands
capacity_factor DACCS Westeros
capacity_factor DACCS Westerlands
technical_lifetime DACCS Westeros
technical_lifetime DACCS Westerlands
growth_new_capacity_up DACCS Westeros
growth_new_capacity_up DACCS Westerlands
initial_new_capacity_up DACCS Westeros
initial_new_capacity_up DACCS Westerlands


{'emission_factor': [    node_loc technology  year_vtg  year_act      mode emission  value  \
  0   Westeros      DACCS       700       700  standard      CO2    -20   
  1   Westeros      DACCS       700       710  standard      CO2    -20   
  2   Westeros      DACCS       700       720  standard      CO2    -20   
  3   Westeros      DACCS       710       710  standard      CO2    -20   
  4   Westeros      DACCS       710       720  standard      CO2    -20   
  5   Westeros      DACCS       720       720  standard      CO2    -20   
  6   Westeros      DACCS       700       700  standard      CH4      0   
  7   Westeros      DACCS       700       710  standard      CH4      0   
  8   Westeros      DACCS       700       720  standard      CH4      0   
  9   Westeros      DACCS       710       710  standard      CH4      0   
  10  Westeros      DACCS       710       720  standard      CH4      0   
  11  Westeros      DACCS       720       720  standard      CH4      0   
  
   

In [10]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in parameters.keys()}

In [ ]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

,node_loc,technology,year_vtg,year_act,mode,emission,value,unit
0,Westeros,DACCS,700,700,standard,CO2,-20,tCO2/kWa
1,Westeros,DACCS,700,710,standard,CO2,-20,tCO2/kWa
2,Westeros,DACCS,700,720,standard,CO2,-20,tCO2/kWa
3,Westeros,DACCS,710,710,standard,CO2,-20,tCO2/kWa
4,Westeros,DACCS,710,720,standard,CO2,-20,tCO2/kWa
5,Westeros,DACCS,720,720,standard,CO2,-20,tCO2/kWa
6,Westeros,DACCS,700,700,standard,CH4,0,tCO2/kWa
7,Westeros,DACCS,700,710,standard,CH4,0,tCO2/kWa
8,Westeros,DACCS,700,720,standard,CH4,0,tCO2/kWa
9,Westeros,DACCS,710,710,standard,CH4,0,tCO2/kWa
